
# Practica tematica libre
### Araceli Macía Barrado


-------------

Para la realización de esta práctica, después de navegar por Internet buscando algo que me interesara, llegue a la página web del Ayuntamiento de Madrid, dado su política de transparencia, en su página web publican muchos datos.

Los datos los he descargado de este link:

http://datos.madrid.es/


Los datos que me han interesados son los relativos a las multas de tráfico.
He descargado todos los ficheros disponibles del 2015, es un fichero por mes, a excepcion del mes de Febrero que no lo han publicado correctamente.

Esta practica tiene dos noteboook:
1. Practica_Final_Multas_1  :  Este notebook, donde leo los ficheros que he descargado de la web, y los preparo y almaceno en un fichero hdf5 para posteriormente analizarlo.
2. Practica_Final_Multas_2 : Notebook donde realizo el analisis de los datos de las multas.

Lo he hecho así debido al volumen de los datos, al separarlo me facilita mucho al bajar los tiempos de respuesta en la realización del analisis.


In [1]:
import numpy as np
import pandas as pd
import glob
import cProfile

## Lectura y preparación de datos.

In [2]:

def ConvMes(x):
    meses=["ENERO","FEBRERO","MARZO","ABRIL","MAYO","JUNIO","JULIO","AGOSTO","SEPTIEMBRE",\
           "OCTUBRE","NOVIEMBRE","DICIEMBRE"]
    
    x=str(x)
    if (x.isdigit()):
        return  int(x) 
    else:
        return int(meses.index(x)+1)

def Texto(x):
    x=x.encode('utf8')
    return x.upper().strip()

def ConVelo(x):
    x=str(x)      #Los Valores de velocidad solo se rellenan en las multas de exceso de velocidad, Cuando no vengan
    if (x.isdigit()):   #rellenos, he decidido poner un 0.
        return  int(x) 
    else: 
        return 0

def CamHora(x):  #Al dato de la hora cambio el caracter de . por : para poder convertirlo a formato tiempo.
    x=str(x)      
    return x.replace(".",":")


Tras realizar varias cargas y visualizar los datos, he ido añadiendo funciones en la carga de los datos, con objetivo de normalizar los datos de los ficheros para poder tratarlos.

In [3]:
ficheros= glob.glob("multas/20*.csv") #almaceno todos los nombres de los 11 ficheros csv que tengo en el directorio.
dataTotal=pd.DataFrame()    
filas=0                     
for fic in ficheros:
    data1= pd.read_csv(fic,index_col=None, header=0, sep=";", \
                   converters={"CALIFICACION":Texto , "HORA":CamHora,\
                                "LUGAR":Texto,"DESCUENTO":Texto,
                                "DENUNCIANTE":Texto,"HECHO-BOL":Texto, "MES":ConvMes, \
                                "VEL_LIMITE":ConVelo,"VEL_CIRCULA":ConVelo },\
                                 encoding="latin1")
    filas=filas+ data1.shape[0]
    dataTotal=dataTotal.append(data1,ignore_index=True)


    

In [4]:
dataTotal=dataTotal.rename(columns={"HECHO-BOL": "HECHO"}) #Renombro esta columna, por el caracter -
%time dataTotal["DHORA"]=pd.to_datetime("2016-01-01 " + dataTotal.HORA,unit="h") #Genero una columna con la hora en formato timestamp

CPU times: user 615 ms, sys: 143 ms, total: 757 ms
Wall time: 878 ms


In [5]:
print dataTotal.shape
print dataTotal.info()
dataTotal.head(1)

(1921501, 13)
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1921501 entries, 0 to 1921500
Data columns (total 13 columns):
CALIFICACION    object
LUGAR           object
MES             int64
ANIO            int64
HORA            object
IMP_BOL         float64
DESCUENTO       object
PUNTOS          int64
DENUNCIANTE     object
HECHO           object
VEL_LIMITE      int64
VEL_CIRCULA     int64
DHORA           datetime64[ns]
dtypes: datetime64[ns](1), float64(1), int64(5), object(6)
memory usage: 205.2+ MB
None


,CALIFICACION,LUGAR,MES,ANIO,HORA,IMP_BOL,DESCUENTO,PUNTOS,DENUNCIANTE,HECHO,VEL_LIMITE,VEL_CIRCULA,DHORA
0,GRAVE,M 30 04FL12,1,2015,15:50,200.0,SI,0,POLICIA MUNICIPAL,ADELANTAR EN ZIGZAG.,0,0,2016-01-01 15:50:00


Realizo una revisión de los diferentes valores de los datos, para comprobar valores invalidos.

In [6]:

print dataTotal.CALIFICACION.unique()
print dataTotal.LUGAR.unique()
print dataTotal.MES.unique()
print dataTotal.DESCUENTO.unique()
print dataTotal.DENUNCIANTE.unique()
print dataTotal.HECHO.unique()
dataTotal.notnull().all()




['GRAVE' 'LEVE' 'MUY GRAVE']
['M 30 04FL12' 'VIRGEN DE LOS PELIGROS 10' 'CASADO DEL ALISAL 9' ...,
 'SAN CLAUDIO 43' 'AV RAFAEL ALBERTI 35' 'CJON RUICES-MARTINEZ RIVA']
[ 1  3  4  5  6  7  8  9 10 11 12]
['SI' 'NO']
['POLICIA MUNICIPAL' 'SER' 'AGENTES DE MOVILIDAD' 'SACE']
['ADELANTAR EN ZIGZAG.' 'ESTACIONAR EN CARRIL DE CIRCULACION.'
 'ESTACIONAR EN LUGAR PROHIBIDO DEBIDAMENTE SE\xc3\x91ALIZADO.' ...,
 'CONDUCCION NEGLIGENTE: ESTACIONADO SIN MEDIDAS QUE EVITEN MOVIMIENTO CON DA\xc3\x91OS'
 'CONDUCCION NEGLIGENTE: NO PONER EL FRENO DE MANO Y COLISIONA CON OTRO VEHICULO'
 'CONDUCCION NEGLIGENTE: NO ADOPTAR LAS MEDIDAS PRODUCIENDO DA\xc3\x91OS']


CALIFICACION    True
LUGAR           True
MES             True
ANIO            True
HORA            True
IMP_BOL         True
DESCUENTO       True
PUNTOS          True
DENUNCIANTE     True
HECHO           True
VEL_LIMITE      True
VEL_CIRCULA     True
DHORA           True
dtype: bool

Voy a clasificar los datos del campo HECHO para poder trabajar con ellos.
Lo voy a realizar "a mano" , aunque se que podria hacerlos con un clasificado tipo Kbeans.


In [7]:
def TipoMulta2(x):

    estacionar=["ESTACI","DESCARGA","DOBLE FILA"]
    cinturon=["CINTURON"]
    drogas=["ALCOHO","DROG"]
    semaforo=["SEMAFO","SEMÁFO"]
    desobediencia=["FUGA","NO OBECEDER","DESOBEDE", "CONTROL", "OMISO", "CASO", "ZAFAR", "NO OBEDECE", "PATRULL", "CORDON POLICIAL",\
                   "SE NIEGA" , "ALTO", "SALTARSE","SE LE INDICA", "SE LE INSTA", "SE LE ORDENA", "SE LE PROHIBE" ,\
                    "SE ORDENA", "SE PERCATA","SE RESISTE", "SE SALTA CORT","NO COMUNICAR"]

    velocidad=["VELOCID",  "CONDUCIR", "MAS DE 100"  ]
    otrosVEH=["CICLO", "BICIC",  "MONOPAT"]

    circulacion=["ATRAVESAR", "CAMBI",  "CIRCUL",  "CONTRARI","ADELANT",  "CONDUCCION", "COLAPSA", "PROHIBIDA", "ZIG", \
                 "COMPETIC", "INVAD", "EMBEST",  "RESPET", "CRUZA", "ENTORPE", "DETENE", "DORMIDO",\
                 "REBASA","REGULA", "SE DESPLAZA", "EQUIVOCA","FRENA", "PARAR EN"]
    peaton=["PEATON"]

    trompos=["CABALLI", "TROMPO", "PATIN", "DERRAP", "DESTELL", "COMPORTARSE", "CHIRRI",  "HACIENDO RUEDAS","TIRAN DEL FRENO DE MANO"]


    otrosNegli=["CLIENTE","CASCO","SIN GRAVEDAD", "REPOSTA", "MOTOR", "RETIRAR", "REVISANDO", "ALUMBRADO", "MEDIDAS NECESARIAS" ,"ESCAYOLA", "COME",\
                "COMIENDO","ACÚSTIC", "TRANSPORT" , "ATASCO", "MOVIL", "ELEMENTOS VEGETALES","NAVEGA","ONA CORTADA", "ZONA EXLCUIDA" , "VIAJER"  ]


    rdo="NO"
    
    if (rdo=="NO"):
        for d1 in desobediencia:
            if d1 in x:
                rdo="DESOBEDIENCIA"
                break
                
    if (rdo=="NO"):            
        for d1 in estacionar:
            if d1 in x:
                rdo="ESTACIONAR"
                break

    if (rdo=="NO"):
        for d1 in trompos:
            if d1 in x:
                rdo="TROMPOS"
                break         
   
    if (rdo=="NO"):
        for d1 in otrosVEH:
            if d1 in x:
                rdo="OTROSVEH"
                break  
                
    if (rdo=="NO"):
        for d1 in cinturon:
            if d1 in x:
                rdo="CINTURON"
                break

    if (rdo=="NO"):
        for d1 in drogas:
            if d1 in x:
                rdo="DROGAS"
                break

    if (rdo=="NO"):
        for d1 in semaforo:
            if d1 in x:
                rdo="SEMAFORO"
                break

    if (rdo=="NO"):
        for d1 in peaton:
            if d1 in x:
                rdo="PEATON"
                break  

    

    if (rdo=="NO"):
        for d1 in velocidad:
            if d1 in x:
                rdo="VELOCIDAD"
                break        
    if (rdo=="NO"):
        for d1 in circulacion:
            if d1 in x:
                rdo="CIRCULACION"
                break 
    
    if (rdo=="NO"):
        for d1 in otrosNegli:
            if d1 in x:
                rdo="VAR_NEGLIGENCIA"
                break   
                
        
       

    if (rdo=="NO"):
          rdo="VAR_TEMERARIA"      

    return rdo

In [8]:
%time dataTotal["TIPO"]=dataTotal["HECHO"].apply(TipoMulta2)

CPU times: user 13.3 s, sys: 198 ms, total: 13.5 s
Wall time: 14.1 s


In [9]:
def TramoHorario(x):  #Funcion para crear tramos horarios a partir de las horas de las multas. 
    horas=["2016-01-01 00:00","2016-01-01 06:00","2016-01-01 08:00",\
           "2016-01-01 10:00","2016-01-01 12:00","2016-01-01 14:00",\
           "2016-01-01 16:00","2016-01-01 18:00","2016-01-01 21:00","2016-01-01 00:00"]
    horas=pd.to_datetime(horas,unit="h")
    if ((x>=horas[0]) & (x<horas[1])):
        return "00"
    elif ((x>=horas[1]) & (x<horas[2])):
        return "06"   
    elif ((x>=horas[2]) & (x<horas[3])):
        return "08"
    elif ((x>=horas[3]) & (x<horas[4])):
        return "10"
    elif ((x>=horas[4]) & (x<horas[5])):
        return "12"
    elif ((x>=horas[5]) & (x<horas[6])):
        return "14"
    elif ((x>=horas[6]) & (x<horas[7])):
        return "16"
    elif ((x>=horas[7]) & (x<horas[8])):
        return "18"
   
    else:
        return "21"  
    
    

In [10]:
%time dataTotal["TRAMOHORA"] = dataTotal["DHORA"].apply(TramoHorario)

CPU times: user 3min 42s, sys: 4.09 s, total: 3min 46s
Wall time: 3min 59s


### Una vez que ya tengo los datos clasificados y en el formato deseado, lo guardo como hdf5, para poder trabajar con ellos, pero eso lo haré en otro Notebook, para facilitar los tiempos de ejecucion en la realizacion del analisis de los datos.

In [11]:
%rm multas/Multas2015.hdf5
import h5py
from pandas import read_hdf
 
store = pd.HDFStore('multas/Multas2015.hdf5',mode='w')
store.append("multas",dataTotal)
store.close()